# Poisson processes for fun and profit

Nelson Uhan<br>
October 2019

## A really important question

Do Taylor Swift's tweets follow a Poisson process?

## How do we determine if an arrival process is Poisson?

One approach: __look at the interarrival times__.

* Are the interarrival times exponentially distributed?
* Are the interarrival times independent?

## Preliminaries

This is a [Jupyter Notebook](http://www.jupyter.org), which lets you mix live code, equations, text, and images into one interactive document. 

The code in this notebook is written in the [Python](http://www.python.org) programming language.

To execute a code cell:

1. Click inside a code cell
2. Either
    * press <key><i class="fa fa-step-forward" aria-hidden="true"></i></key> in the toolbar, or
    * press Shift + Enter

First, we need to import a whole bunch of libraries, including [Tweepy](http://www.tweepy.org), which allows us to interface with Twitter programmatically using Python.

In [ ]:
# Setup
# Import libraries
import tweepy
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display

Next, we need to authenticate into Twitter.

In [ ]:
# Authenticate into Twitter
consumer_key = 'CONSUMER_KEY'
consumer_secret = 'CONSUMER_SECRET'
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

## Getting someone's tweets

Let's grab some information about the Twitter user we want to study.

In [ ]:
# Enter Twitter user name
username = 'taylorswift13'

In [ ]:
# Get information about this Twitter user
user = api.get_user(username)
print(f'Name: {user.name}')
IPython.display.Image(user.profile_image_url)

Let's grab this user's last 200 tweets.

In [ ]:
# Get user's last 200 tweets
public_tweets = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name=username).items(200):
    public_tweets.append(tweet)

Just to make sure we're doing this right &mdash; let's examine this user's last 10 tweets.

In [ ]:
# Print user's last 10 tweets: date/time, text
for tweet in public_tweets[:10]:
    print("{0} {1}".format(tweet.created_at, tweet.text))

## Computing interarrival times

OK, looks good! Now let's create a list of just the tweet arrival times.

In [ ]:
# Grab just the arrival times
arrival_times = []
for tweet in public_tweets:
    arrival_times.append(tweet.created_at)

Next, we can compute the interarrival times by

* sorting the arrival times, and then 
* computing the difference in consecutive arrival times.

The times are in seconds, so we divide the interarrival times by $60 \times 60$ to obtain times in hours.

In [ ]:
# Sort arrival times
arrival_times.sort()

# Compute interarrival times in hours
interarrival_times = []
for a, b in zip(arrival_times, arrival_times[1:]):
    interarrival_times.append((b - a).seconds / (60 * 60))

Another sanity check: do the interarrival times look reasonable? Let's print out the first 10:

In [ ]:
print(interarrival_times[:10])

## Choosing $\lambda$

We want to see if the interarrival times fit an exponential distribution. 

Recall that the exponential distribution has a parameter, the arrival rate $\lambda$. 

What should we use for $\lambda$?

It turns out that the maximum likelihood estimator is

$$
\hat{\lambda} = \frac{1}{\text{sample mean of the observed interarrival times}}
$$

Let's compute this next.

In [ ]:
# Sample mean of the interarrival times
interarrival_time_SM = np.mean(interarrival_times)
print(f'Sample mean of interarrival times: {interarrival_time_SM} hours per tweet')

# Estimated arrival rate (maximum likelihood)
estimated_arrival_rate = 1 / interarrival_time_SM
print(f'Estimated arrival rate: {estimated_arrival_rate} tweets per hour')

Let's compare the histogram of interarrival times with the pdf of the exponential distribution with the estimated arrival rate $\hat{\lambda}$:

In [ ]:
# Create pdf data points
x_max = max(interarrival_times)
x_values = np.arange(0, x_max, x_max / 1000)
pdf_values = [stats.expon.pdf(x, scale=1 / estimated_arrival_rate) for x in x_values]

# Create histogram with pdf of fitted exponential distribution
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(interarrival_times, density=True)
ax.plot(x_values, pdf_values);
ax.set_xlim(0, max(interarrival_times));
ax.set_xlabel('Interarrival time');
ax.set_ylabel('Frequency');
ax.set_title('Histogram of interarrival times and pdf of fitted exponential distribution');

__What do you think?__

__Do you think the interarrival times are from an exponential distribution?__

Ideally, we would perform some goodness-of-fit tests to statistically determine whether the exponential distribution is a good fit for the interarrival times.

We also need to check independence of the interarrival times. One easy visual test is to plot the interarrival times as a time series:

In [ ]:
# Plot interarrival times as time series
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(interarrival_times);
ax.set_xlabel('First to last');
ax.set_ylabel('Interarrival time');
ax.set_title('Interarrival times');

__What do you think?__

__Are the interarrival times independent?__

__What type of user would you expect to tweet according to a Poisson process?__

## #poisson?

We can also do the same thing with hashtags. Let's grab the last 200 tweets with a certain hashtag.

In [ ]:
# Enter hashtag to search
search_text = "#dank"

In [ ]:
# Get last 200 tweets with this hashtag
cursor = tweepy.Cursor(api.search, q=search_text)
hashtag_tweets = []
for tweet in cursor.items(200):
    hashtag_tweets.append(tweet)

In [ ]:
# Print last 10 tweets with this hashtag: date/time, text
for tweet in hashtag_tweets[:10]:
    print("{0} {1}".format(tweet.created_at, tweet.text))

Now, we can go through the same process as we did above.

Since hashtags appear more frequently than one user's tweets, let's change the time scale to minutes instead of hours.

In [ ]:
# Grab just the arrival times
ht_arrival_times = []
for tweet in hashtag_tweets:
    ht_arrival_times.append(tweet.created_at)

# Sort arrival times
ht_arrival_times.sort()

# Compute interarrival times in minutes
ht_interarrival_times = []
for a, b in zip(ht_arrival_times, ht_arrival_times[1:]):
    ht_interarrival_times.append((b - a).seconds / 60)
    
# Sample mean of the interarrival times
ht_interarrival_time_SM = np.mean(ht_interarrival_times)
print("Sample mean of interarrival times: {0} minutes per tweet".format(ht_interarrival_time_SM))

# Estimated arrival rate (maximum likelihood)
ht_estimated_arrival_rate = 1 / ht_interarrival_time_SM
print("Estimated arrival rate: {0} tweets per minute".format(ht_estimated_arrival_rate))

# Create pdf data points
x_max = max(ht_interarrival_times)
x_values = np.arange(0, x_max, x_max / 1000)
pdf_values = [stats.expon.pdf(x, scale=1 / ht_estimated_arrival_rate) for x in x_values]

# Create histogram with pdf of fitted exponential distribution
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(ht_interarrival_times, density=True)
ax.plot(x_values, pdf_values);
ax.set_xlim(0, max(ht_interarrival_times));
ax.set_xlabel('Interarrival time');
ax.set_ylabel('Frequency');
ax.set_title('Histogram of interarrival times and pdf of fitted exponential distribution');

# Plot interarrival times as time series
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(ht_interarrival_times);
ax.set_xlabel('First to last');
ax.set_ylabel('Interarrival time');
ax.set_title('Interarrival times');

__What do you think?__

__Are the interarrival times from an exponential distribution?__

__Are the interarrival times independent?__

__What type of hashtag would you expect to follow a Poisson process?__